In [1]:
import numpy as np
import pandas as pd
from Func import minimization
from Func import callback
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.__version__
pd.__version__

'1.3.5'

In [2]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [3]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['region_scenario','regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])
df

,,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
region_scenario,regionbase,,,,,,,,,,,,,,,
1,US as Home,4.480045,27.795946,32.275990,4.598403,1.196111,0.421601,26.059874,5.794514,26.481474,5.020004,27.255985,32.275990,32.275990,0.015921,0.793579
2,EU28 as Home,0.935781,31.340202,32.275986,2.950581,1.013552,0.507676,27.804173,3.964133,28.311848,3.458257,28.817724,32.275982,32.275982,0.017932,0.744319
3,OECD37 as Home,8.625495,23.650503,32.276001,11.293668,2.487537,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
4,World as Home,32.175999,0.100000,32.276001,32.146000,0.030000,0.030000,0.070000,32.175999,0.100000,32.175999,0.100000,32.276001,32.276001,0.300000,0.999068
5,China as Home,7.522744,24.753246,32.275990,7.345464,0.632472,1.935382,22.362669,7.977936,24.298052,9.280846,22.995142,32.275990,32.275990,0.079652,0.920722
6,OECD and China as Home,16.148245,16.127758,32.276001,20.103180,1.655961,1.381915,9.134943,21.759140,10.516858,21.485096,10.790904,32.276001,32.276001,0.131400,0.923896
7,US and EU as Home,5.415825,26.860159,32.275986,7.728876,2.029770,0.749387,21.767950,9.758646,22.517336,8.478263,23.797718,32.275982,32.275982,0.033280,0.792003


In [4]:
## choose which regional scenario to run (runs all if not executed)
#df=df.drop([1,2,4,5,6,7])  

In [5]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 2  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [12]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[ 2, 3, 4, 5, 6, 7, 8, 9])
# use for quick test: tax_scenario= pd.DataFrame({'tax_sce': 'EPC_hybrid', 'Base':1},index=[1])
#tax_scenario= pd.DataFrame({'tax_sce': 'Unilateral', 'Base':0},index=[1])

In [13]:
def cases(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    # varphilist = np.arange (1.6,2.5,0.1) # marginal damages
    varphilist = np.arange (0,50,0.2) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    varphilist = [2]
    output=[]
    for varphi in varphilist:    
        te = 0; #initial value of extraction tax for iteration
        tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)

        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(minimization, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
        
        appended_df = pd.merge(df, tax_df, on=['region_scenario','regionbase'])

        ## back out other optimal values
        output_df = appended_df.apply(callback, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
        output_df = pd.merge(tax_df, output_df, on=['region_scenario','regionbase'])
        output.append(output_df)
        print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb

    print(tax_scenario['tax_sce'])
    return output


In [14]:
## apply the above for each tax scenario
output_all=tax_scenario.apply(cases, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))

2
Unilateral
2
purete
2
puretc
2
puretp
2
EC_hybrid
2
EP_hybrid
2
PC_hybrid
2
EPC_hybrid


In [16]:
output_list=[]
for i in range(2,len(tax_scenario)+1):
    output_list.append(output_all.loc[i])

In [17]:
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)

In [18]:
Outcomes

,tax_sce,region_scenario,tb,prop,te,varphi,pe,jxbar_prime,jmbar_prime,j0_prime,...,leakage3,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality,prop2
regionbase,,,,,,,,,,,,,,,,,,,,,
US as Home,Unilateral,1,1.336634,-132.738628,2.000000,2.0,0.980111,0.019923,0.793579,0.009563,...,0.078606,-1.094690,-0.259121,0.240012,-3.053360,-3.700940,0.169824,9.381382,-6.426811,NaN
EU28 as Home,Unilateral,2,1.354922,-134.516400,2.000000,2.0,0.975482,0.022563,0.744319,0.010712,...,0.181227,-1.517988,-0.014206,0.347210,-1.915878,-4.502759,0.172423,8.973335,-5.525760,NaN
OECD37 as Home,Unilateral,3,1.372099,-136.147458,2.000000,2.0,0.929549,0.062700,0.819498,0.029179,...,0.097206,-3.215026,-0.341964,0.690474,-7.103192,-7.901685,0.181277,9.256999,-6.205783,NaN
World as Home,Unilateral,4,1.336454,-132.643433,2.000000,2.0,0.869477,0.359848,0.999068,0.196881,...,-0.000522,-0.024401,-0.012775,-0.009187,-17.599056,-0.241186,0.187363,9.075760,-7.333108,NaN
China as Home,Unilateral,5,1.339212,-132.921188,2.000000,2.0,0.965720,0.098576,0.920722,0.048844,...,-0.103227,-1.667974,-0.040438,-0.519914,-5.036607,-16.836950,0.172194,11.063123,-7.876388,NaN
OECD and China as Home,Unilateral,6,1.406837,-139.839900,2.000000,2.0,0.865576,0.168670,0.923896,0.078150,...,0.035645,-4.044486,0.038579,0.419346,-11.764449,-11.722443,0.196011,9.536643,-6.683367,NaN
US and EU as Home,Unilateral,7,1.362792,-135.279199,2.000000,2.0,0.951082,0.042143,0.792003,0.019794,...,0.120511,-2.563605,-0.242767,0.595957,-4.945245,-6.563999,0.176909,9.174536,-6.028120,NaN
US as Home,purete,1,0.000000,1.000000,0.704250,2.0,1.022255,0.015921,0.793579,0.015921,...,-0.820467,1.250983,-0.593385,-0.576522,-0.702676,-0.062552,0.000000,1.696878,-1.941092,NaN
EU28 as Home,purete,2,0.000000,1.000000,0.674188,2.0,1.004194,0.017932,0.744319,0.017932,...,-0.877147,0.263417,-0.120349,-0.118236,-0.134796,-0.014194,0.000000,0.466694,-0.553425,NaN


In [12]:
Outcomes.to_csv('../output/old_output.csv'.format(case), header=True) 

In [ ]:
if epsilonS == 0.5 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}.csv'.format(case), header=True) 
elif epsilonS == 0.5 and epsilonSstar == 2:
    Outcomes.to_csv('output/output_case{0}_D_2.csv'.format(case), header=True) 
elif epsilonS == 2 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}_2_D.csv'.format(case), header=True) 

In [11]:
temp = pd.merge(Outcomes, df, on = ['region_scenario'])
temp.head()

,tax_sce,region_scenario,tb,prop,te,varphi,pe,jxbar_prime,jmbar_prime,j0_prime,...,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
0,Unilateral,3,3.576279e-10,1.000000,0.0,0.0,1.000000,0.049234,0.819498,0.049234,...,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
1,Unilateral,3,7.957103e-02,-6.969603,0.2,0.2,0.988370,0.049307,0.819498,0.047105,...,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
2,Unilateral,3,1.580208e-01,-14.776593,0.4,0.4,0.980651,0.049511,0.819498,0.045204,...,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
3,Unilateral,3,2.361444e-01,-22.595227,0.6,0.6,0.976504,0.049826,0.819498,0.043496,...,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
4,Unilateral,3,3.147025e-01,-30.432747,0.8,0.8,0.975792,0.050239,0.819498,0.041951,...,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498


In [16]:
from scipy.integrate import quad

In [24]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
temp.apply(verify, axis = 1)

0.0 -7.105427357601002e-15 3.576278269541946e-10
0.2 -3.552713678800501e-15 -8.550048502531382e-09
0.4 -8.526512829121202e-14 -4.205375542243317e-08
0.6000000000000001 -3.623767952376511e-13 -5.452795143190414e-08
0.8 -4.369837824924616e-13 4.6895619609799155e-09
1.0 -1.7053025658242404e-13 -1.9319830069086663e-08
1.2000000000000002 -1.0658141036401503e-14 -5.074265729820837e-08
1.4000000000000001 7.105427357601002e-15 -3.070851906095129e-08
1.6 -7.105427357601002e-15 -4.109534113005964e-08
1.8 -3.552713678800501e-15 -1.9208017398852917e-09
2.0 -7.105427357601002e-15 -8.448091315393924e-08
2.2 0.0 -8.449591371029186e-08
2.4000000000000004 3.552713678800501e-15 -1.1470213912012639e-07
2.6 -9.059419880941277e-13 -2.4232438056159822e-08
2.8000000000000003 -8.171241461241152e-14 -1.0815285000909114e-07
3.0 -7.105427357601002e-15 6.173740407788841e-08
3.2 3.552713678800501e-15 -2.5370210821051842e-08
3.4000000000000004 -3.552713678800501e-15 -8.907053472562154e-08
3.6 0.0 -3.655910862931932

0      1.278977e-19
1      7.310333e-17
2      1.768518e-15
3      2.973297e-15
4      2.199199e-17
           ...     
245    1.345051e+02
246    1.389570e+02
247    1.434814e+02
248    1.480783e+02
249    1.527477e+02
Length: 250, dtype: float64

In [23]:
def verify(df):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
    tax_scenario = {'tax_sce': 'Unilateral', 'Base': 0}
    varphi = df['varphi']
     #solve for equilibrium
    pe = df['pe']
    tb_mat = [df['tb'], df['prop']]
    te = 0
    #print('input is:', p)
    #print(te,varphi, tax_scenario, ParaList, df)
    # print('pe='+str(pe))
    if tax_scenario['tax_sce']=='purete':
        print("shouldn't be here")
        return (1,1,1,1)
        
    # if pe+tb_mat[0]-tb_mat[1]<=0 and tax_scenario['tax_sce']=='EP_hybrid':
    #     # print(tb_mat)
    #     # print(pe)
    #     # tb_mat[0]=tb_mat[1]-pe;
    #     tb_mat[1]=pe+tb_mat[0];
    #     te=tb_mat[1];
 
    
    # print(tb_mat)
     ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * tb_mat[0]/pe) ** (-theta) * (1 + tb_mat[0]/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
    j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta))
    jmbar_hat = 1
    #print('j0 hat is:', j0_hat)
    #print('denom is:', (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta)))
    #print('num is:', (pe+tb_mat[0])**(-(1-alpha)*theta))
    #print('first part is:', df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta))
    
    if tax_scenario['tax_sce']=='Unilateral':
        te=varphi;
        tb_mat[1]=1;
    
    
    jxbar_prime = jxbar_hat * df['jxbar'];
    jmbar_prime = jmbar_hat * df['jmbar'];
    
    j0_prime = j0_hat * df['jxbar'];

    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    Bfunvec1_prime = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))[0];
    Bfunvec2_prime = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))[0];

    #if te is too large, Home stop producing
    petbte = pe + tb_mat[0] - te
    z = pe + tb_mat[0] >= te
    petbte = petbte * z

    Qe_hat = (petbte) ** epsilonS
    Qestar_hat = pe ** epsilonSstar
          
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta);
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta);
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta);
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta);
 
    Qe_prime = df['Qe'] * Qe_hat;
    Qestar_prime = df['Qestar'] * Qestar_hat;
     
    CeHH_hat = (pe + tb_mat[0]) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + tb_mat[0]) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +tb_mat[0])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + tb_mat[0]/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
        
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + tb_mat[0]) ** (-epsilonD);
    
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar);
    
    ##
    VgHH = df['CeHH']/(1 - alpha);
    VgFF = df['CeFF']/(1 - alpha);
    
    VgFH = df['CeFH'] /(1 - alpha);
    # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta));
    VgFH1_hat = (pe + tb_mat[0]) * CeFH1_hat;
    VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta));
    VgFH1_prime = VgFH * VgFH1_hat;
    VgFH2_prime = VgFH * VgFH2_hat;
    
    

    if tax_scenario['tax_sce'] == 'Unilateral':
        diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime);
        pai_g = (pe + tb_mat[0]) * CeFH2_prime / (1 - alpha) - VgFH2_prime
    
        numerator = varphi * epsilonSstar * Qestar_prime - sigmastar * (1-alpha) * pai_g
        denominator = epsilonSstar * Qestar_prime + epsilonDstar * CeFF_prime
        diff1 = tb_mat[0] - numerator/denominator
        print(varphi, diff,diff1)
        return diff**2 + diff1**2